In [1]:
# Import Dependencies
import pandas as pd
import json
import requests
from datetime import date, timedelta, datetime
from config import api_key

In [2]:
# Save config information
url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"
lat = 41.881832
lon = -87.623177

In [3]:
#Define start and end dates, create a date list and a loop to add all desired dates to the date list
start_date = date(2022, 1, 1) 
end_date = date(2022, 12, 31)
current_date = start_date
date_list = []

while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date+=timedelta(days=1)

In [4]:
#Create a for loop to retrieve OpenWeather data for all dates in the date list

#Set up response info lists
cloud_cover = []
precipitation = []
min_temp = []
max_temp = []
temp_mrng = []
temp_aft = []
temp_eve = []
temp_night = []
wind_max = []

# Loop through all the cities in our list to fetch weather data
for day in date_list:
        
# Create endpoint URL with each day
    query_url = f"{url}&lat={lat}&lon={lon}&date={day}&appid={api_key}&q="

    response = requests.get(query_url).json()
    cloud_cover.append(response['cloud_cover']['afternoon'])
    precipitation.append(response['precipitation']['total'])
    min_temp.append(response['temperature']['min'])
    max_temp.append(response['temperature']['max'])
    temp_mrng.append(response['temperature']['morning'])
    temp_aft.append(response['temperature']['afternoon'])
    temp_eve.append(response['temperature']['evening'])
    temp_night.append(response['temperature']['night'])
    wind_max.append(response['wind']['max']['speed'])


In [7]:
weather_dict = {
    "date": date_list,
    "cloud_cover": cloud_cover,
    "precipitation": precipitation,
    "min_temp": min_temp,
    "max_temp": max_temp,
    "morning_temp": temp_mrng,
    "afternoon_temp": temp_aft,
    "evening_temp": temp_eve,
    "night_temp": temp_night,
    "max_windspeed": wind_max
}

weather_df = pd.DataFrame(weather_dict)
weather_df.head()

,date,cloud_cover,precipitation,min_temp,max_temp,morning_temp,afternoon_temp,evening_temp,night_temp,max_windspeed
0,2022-01-01,90.0,0.18,273.83,278.76,278.76,276.72,274.95,276.73,6.71
1,2022-01-02,90.0,13.81,265.82,273.58,271.06,270.17,266.28,273.58,8.23
2,2022-01-03,16.0,0.00,259.03,270.02,267.46,262.18,259.38,268.82,4.02
3,2022-01-04,4.0,0.00,265.39,272.24,265.39,267.02,267.20,266.07,4.47
4,2022-01-05,64.0,0.25,262.91,273.92,272.89,273.56,267.70,272.95,7.60


In [9]:
#Export our weather dataframe to a csv file
weather_df.to_csv("../Resources/weather_daily.csv", index=False, header=True)